In [ ]:
#Local
try:
    from kubeflow import fairing

#Container
except:
    !pip install kubeflow-fairing
    from kubeflow import fairing

In [ ]:
import sys
py_version = ".".join([str(x) for x in sys.version_info[0:3]])
base_image = 'gcr.io/kubeflow-images-public/tensorflow-1.14.0-notebook-cpu:v0.7.0'

In [ ]:
REGISTRY = 'YOUR REGISTRY'
DOCKER_REGISTRY = f'{REGISTRY}.azurecr.io'
fairing.config.set_builder('append',base_image=base_image, registry=DOCKER_REGISTRY, push=True)
fairing.config.set_deployer('job')

In [ ]:
def train():
    print("hello world!")

In [ ]:
!az acr login --name {REGISTRY}

In [ ]:
!docker login -u USERNAME -p PASS REGISTRY

In [ ]:


remote_train = fairing.config.fn(train)
remote_train()



In [ ]:
import tensorflow as tf


In [ ]:
class trymodel(object):
    
    
    def __init__(self):
        self.x,self.y = data,target = make_regression(n_samples=1000,n_features=1,bias=5,n_informative=1,random_state=1)
        self.w = None
        self.model = False
        
    def train(self, x=None, y=None):
        init = tf.global_variables_initializer()
        sess = tf.Session()
        sess.run(init)
        
        if x==None or y==None:
            x,y = self.x,self.y
        
        ones = np.ones((x.shape[0],1))
        
        x = np.append(ones,x,axis=1)

        XtX = tf.matmul(x,x,transpose_a=True)
        XtY = tf.matmul(x,y.reshape(1000,1),transpose_a=True)

        w = tf.matmul(tf.matrix_inverse(XtX),XtY)
        self.w = np.array(sess.run(w))
        self.model=True
        
    def predict(self,x):
        if self.model==False:
            self.train()
            self.model=True
            
        sess = tf.Session()
        x = np.array(x)
        x = x.reshape((-1,1))
        x = np.append(np.ones((x.shape[0],1)),x,axis=1)
    
        return sess.run(tf.tensordot(x,self.w,1))
    

In [ ]:
m = trymodel()
m.train()

In [ ]:
pred = fairing.PredictionEndpoint(trymodel,docker_registry=DOCKER_REGISTRY,base_docker_image=base_image)

In [ ]:
pred.create()